In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import sys

sys.path.append("../")

from src.envs import RLFSEnvSparse
from src.rl import REINFORCE
from src.errors import sammon_error

In [ ]:
INF_LOOP_CNT = 5

In [ ]:
def get_data_frames(train_path, test_path):
    data_train = pd.read_csv(train_path, sep=",")
    data_test = pd.read_csv(test_path, sep=",")

    return data_train, data_test


def get_data_train_test(data_train, data_test):
    X_train = data_train.drop(columns=["repository"], inplace=False)
    X_train = X_train.to_numpy()

    X_test = data_test.drop(columns=["repository"], inplace=False)
    X_test = X_test.to_numpy()

    return X_train, X_test

def powers_of_two_less_than(n):
    max_exponent = int(np.log2(n))  # Find the largest exponent such that 2^k < N
    return 2 ** np.arange(max_exponent+1)

In [ ]:
def train_reinforce_agent(X_train, agent, device, make_plots=False):
    state_space = X_train.shape[1]
    action_space = X_train.shape[1]
    for i, num_features in enumerate(powers_of_two_less_than(state_space//2)):
            env = RLFSEnvSparse(
                state_size=state_space, data=X_train, max_features=num_features
            )
            print(f"Iteration {i}, Number of features {num_features}")
            episode_returns = agent.train(
                env=env,
                num_episodes=500 + 300 // (i+1),
                max_steps=num_features,
            )
            # torch.save(agent.policy.state_dict(), "models/REINFORCE/policy_weights.pth")
            if make_plots:
                plt.plot(episode_returns)
                plt.show()

In [ ]:
def test_agent(X_test, data_test, agent, verbose=True):
    state_space = X_test.shape[1]
    action_space = X_test.shape[1]
    env = RLFSEnvSparse(state_size=state_space, data=X_test, max_features=state_space)
    errors = []
    num_ftrs = []
    selected_features = []
    if verbose:
        print("waiting...")
    for n in range(0, state_space+1):
        state = env.reset()
        # errors.append(sammon_error(X_test, state))
        state_cnt = 0  # int(np.sum(state))
        done = False
        # if verbose:
        #     print(f"n={n}")
        inf_loop_cnt = INF_LOOP_CNT
        action_sequence = []
        while state_cnt < n and not done:
            if inf_loop_cnt > 0:
                action, action_prob = agent.select_action_deterministic(state)
            else:
                # print("+")
                action, action_prob = agent.select_action(state)
                # print(np.exp(action_prob.detach().numpy()))

            next_state, _, done, _ = env.step(action)
            action_sequence.append(action)

            if int(np.sum(next_state)) > state_cnt:
                # print(state_cnt)
                inf_loop_cnt = INF_LOOP_CNT
                state_cnt = int(np.sum(next_state))
            else:
                inf_loop_cnt -= 1

            state = next_state
        # print(np.sum(state))
        error = sammon_error(X_test, state)
        errors.append(error)
        num_ftrs.append(n)
        selected_features.append(
            [data_test.drop(columns=["repository"]).columns[action] for action in action_sequence]
        )

    return errors, num_ftrs, selected_features

In [ ]:
train_path_class = "../data/data_class_train.csv"
test_path_class = "../data/data_class.csv"
train_path_method = "../data/data_method_train.csv"
test_path_method = "../data/data_method.csv"

In [ ]:
data_train_method, data_test_method = get_data_frames(train_path_method, test_path_method)
X_train_method, X_test_method = get_data_train_test(data_train_method, data_test_method)
state_space_method = X_train_method.shape[1]
action_space_method = X_train_method.shape[1]

data_train_class, data_test_class = get_data_frames(train_path_class, test_path_class)
X_train_class, X_test_class = get_data_train_test(data_train_class, data_test_class)
state_space_class = X_train_class.shape[1]
action_space_class = X_train_class.shape[1]

In [ ]:
data_train_method.shape, data_test_method.shape

In [ ]:
data_train_class.shape, data_test_class.shape

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = REINFORCE.REINFORCEAgent(state_space_method, action_space_method, gamma=1, lr=0.0001)

In [ ]:
def run_train_test(X_train, X_test, data_test, agent, device):
    train_reinforce_agent(X_train, agent, device)
    return test_agent(X_test, data_test, agent)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
agent_method = REINFORCE.REINFORCEAgent(
    state_space_method, action_space_method, gamma=1, lr=0.0001
)
errors, num_ftrs, selected_features = run_train_test(
    X_train=X_train_method,
    X_test=X_test_method,
    data_test=data_test_method,
    agent=agent_method,
    device=device,
)

print(selected_features)


In [ ]:
selected_features_ = ['']
selected_features_.extend(selected_features[-1])

In [ ]:
# Plot
plt.figure(figsize=(10, 7.5))
plt.plot(num_ftrs, errors, marker='o', linestyle='-', color='b')
plt.xlabel('Sequentially Selected Metrics')
plt.ylabel('Sammon Error')
# plt.title('Sammon Error vs Number of Features')
plt.xticks(num_ftrs, selected_features_, rotation=45, ha='right')  # Map indices to feature names
plt.grid(True)
plt.show()

In [ ]:
agent_class = REINFORCE.REINFORCEAgent(
    state_space_class, action_space_class, gamma=1, lr=0.0001
)
errors, num_ftrs, selected_features = run_train_test(
    X_train=X_train_class,
    X_test=X_test_class,
    data_test=data_test_class,
    agent=agent_class,
    device=device,
)

In [ ]:
selected_features_ = ['']
selected_features_.extend(selected_features[-1])

In [ ]:
print(selected_features_)

In [ ]:
# Plot
plt.figure(figsize=(10, 7.5))
plt.plot(num_ftrs, errors, marker='o', linestyle='-', color='b')
plt.xlabel('Sequentially Selected Metrics')
plt.ylabel('Sammon Error')
# plt.title('Sammon Error vs Number of Features')
plt.xticks(num_ftrs, selected_features_, rotation=45, ha='right')  # Map indices to feature names
plt.grid(True)
plt.show()

In [ ]:
def test_stability(X_train, X_test, data_test, num_iters=10):
    state_space = X_train.shape[1]
    action_space = X_train.shape[1]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    all_errors = []
    all_num_ftrs = []
    all_selected_features = []
    for i in range(num_iters):
        agent = REINFORCE.REINFORCEAgent(state_space, action_space, gamma=1, lr=0.0001)
        errors, num_ftrs, selected_features = run_train_test(
            X_train, X_test, data_test, agent, device
        )
        all_errors.append(errors)
        all_num_ftrs.append(num_ftrs)
        all_selected_features.append(selected_features)

    return all_errors, all_num_ftrs, all_selected_features

In [ ]:
all_errors, all_num_ftrs, all_selected_features = test_stability(X_train_class, X_test_class, data_test_class, num_iters=10)

In [ ]:
min_errors = np.min(all_errors, axis=0)
max_errors = np.max(all_errors, axis=0)
avg_errors = np.mean(all_errors, axis=0)
min_errors, max_errors, avg_errors

In [ ]:
df_reinforce_class = pd.DataFrame({
    "number_of_features": all_num_ftrs[0],
    "min_error": min_errors,
    "max_error": max_errors,
    "avg_error": avg_errors
})
df_reinforce_class

In [ ]:
df_reinforce_class.to_csv("../data/results/REINFORCE_class.csv", index=False)

In [ ]:
# Plot
plt.figure(figsize=(10, 7.5))
plt.plot(all_num_ftrs[0], min_errors, marker='o', label='Minimum Error')
plt.plot(all_num_ftrs[0], max_errors, marker='o', label='Maximum Error')
plt.plot(all_num_ftrs[0], avg_errors, marker='o', label='Average Error')

# Add plot details
plt.xlabel('Number of Metrics')
plt.ylabel('Sammon Error')
# plt.title('Model Stability across Retrainings')
plt.xticks(all_num_ftrs[0])  # Ensure x-axis ticks are integers from all_num_ftrs
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
all_errors_m, all_num_ftrs_m, all_selected_features_m = test_stability(X_train_method, X_test_method, data_test_method, num_iters=10)

In [ ]:
print(all_selected_features[0][-1])
print([float(error) for error in all_errors[0]])

In [ ]:
print(all_selected_features_m[0][-1])
print([float(error) for error in all_errors_m[0]])

In [ ]:
min_errors = np.min(all_errors_m, axis=0)
max_errors = np.max(all_errors_m, axis=0)
avg_errors = np.mean(all_errors_m, axis=0)
min_errors, max_errors, avg_errors

In [ ]:
df_reinforce_method = pd.DataFrame({
    "number_of_features": all_num_ftrs_m[0],
    "min_error": min_errors,
    "max_error": max_errors,
    "avg_error": avg_errors
})
df_reinforce_method

In [ ]:
df_reinforce_method.to_csv("../data/results/REINFORCE_method.csv", index=False)

In [ ]:
# Plot
plt.figure(figsize=(10, 7.5))
plt.plot(all_num_ftrs_m[0], min_errors, marker='o', label='Minimum Error')
plt.plot(all_num_ftrs_m[0], max_errors, marker='o', label='Maximum Error')
plt.plot(all_num_ftrs_m[0], avg_errors, marker='o', label='Average Error')

# Add plot details
plt.xlabel('Number of Metrics')
plt.ylabel('Sammon Error')
# plt.title('Model Stability across Retrainings')
plt.xticks(all_num_ftrs_m[0])  # Ensure x-axis ticks are integers from all_num_ftrs
plt.legend()
plt.grid(True)
plt.show()